In [1]:
%matplotlib inline
import sys
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [6]:
# Загрузим словарь со словами объединенной коллекции
cooc_modality = "text"

vocab_list = list(map(lambda r: r[0], filter(lambda r: len(r) > 1 and r[1] == cooc_modality,
                                             map(lambda r: r.strip().split(), open("merged_vocab.txt", "r")))))
vocab_map = dict(zip(vocab_list, range(len(vocab_list))))

In [9]:
%%time

docs_length = 0
word_count = {}

cooc_window_size = 10

# Сначала посчитаем word_count

with open("ruwiki.filtered.csv") as infile:
    reader = csv.reader(infile)
    for i, (title, text) in enumerate(reader, 1):
        tokens = text.split()
        docs_length += len(tokens)
        for token in tokens:
            idx = vocab_map.get(token)
            if idx is None:
                continue
            word_count.setdefault(idx, 0)
            word_count[idx] += cooc_window_size
        if i % 100000 == 0:
            print("Processed %d documents" % i)

Processed 100000 documents
Processed 200000 documents
Processed 300000 documents
Processed 400000 documents
Processed 500000 documents
Processed 600000 documents
Processed 700000 documents
Processed 800000 documents
Processed 900000 documents
Processed 1000000 documents
Processed 1100000 documents
Processed 1200000 documents
Processed 1300000 documents
CPU times: user 7min 13s, sys: 3.2 s, total: 7min 16s
Wall time: 7min 35s


In [11]:
%%time

pair_count = {}

# Теперь считаем pair_count на подвыборке словаря

with open("ruwiki.filtered.csv") as infile:
    reader = csv.reader(infile)
    for i, (title, text) in enumerate(reader, 1):
        tokens = text.split()
        for start in range(len(tokens)):
            start_vocab_id = vocab_map.get(tokens[start])
            if start_vocab_id is None or word_count[start_vocab_id] < 100 * cooc_window_size:
                continue
            end = min(start + cooc_window_size, len(tokens))
            for idx in range(start + 1, end):
                vocab_id = vocab_map.get(tokens[idx])
                if vocab_id is None or word_count[vocab_id] < 100 * cooc_window_size:
                    continue
                pair_id = (min(vocab_id, start_vocab_id), max(vocab_id, start_vocab_id))
                pair_count.setdefault(pair_id, 0)
                pair_count[pair_id] += cooc_window_size - idx + start
        if i % 1000 == 0:
            print("Processed %d documents, len = %d" % (i, len(pair_count)))

Processed 1000 documents, len = 3748690
Processed 2000 documents, len = 6325865
Processed 3000 documents, len = 7460179
Processed 4000 documents, len = 7863882
Processed 5000 documents, len = 9599049
Processed 6000 documents, len = 11126071
Processed 7000 documents, len = 12206100
Processed 8000 documents, len = 13303168
Processed 9000 documents, len = 14332767
Processed 10000 documents, len = 15523009
Processed 11000 documents, len = 16612383
Processed 12000 documents, len = 17484078
Processed 13000 documents, len = 18454529
Processed 14000 documents, len = 19213306
Processed 15000 documents, len = 19747937
Processed 16000 documents, len = 20475850
Processed 17000 documents, len = 21128628
Processed 18000 documents, len = 21720216
Processed 19000 documents, len = 22368280
Processed 20000 documents, len = 23169595
Processed 21000 documents, len = 23735564
Processed 22000 documents, len = 24158593
Processed 23000 documents, len = 24738041
Processed 24000 documents, len = 24981393
Proces

In [33]:
def ppmi_func(n_uv, n_u, n_v, n):
    return max(0, np.log(n_uv * n / n_u / n_v))

In [34]:
%%time

with open("merged_cooc.txt", "w") as cooc_f:
    for (u, v), n_uv in pair_count.items():
        n_u = word_count[u]
        n_v = word_count[v]
        ppmi = ppmi_func(n_uv, n_u, n_v, docs_length)
        cooc_f.write("%d %d %d %.6f\n" % (u, v, n_uv, ppmi))

CPU times: user 9min 51s, sys: 3.9 s, total: 9min 55s
Wall time: 9min 56s


---